## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


______

###### Данное исследование разделим на несколько частей.

##### Часть 1. Изучение общей информации:
* [1. Изученеие файлов с данными, получение общей информации, загрузка библиотек.](#1-bullet)
* [2. Нахождение и ликвидация пропусков.](#2-bullet)

##### Часть 2. Подготовка данных:
* [1. Приведение данных к нужным типам.](#3-bullet)
* [2. Нахождение и исправление аномалий и ошибок.](#4-bullet)
* [3. Категоризация данных.](#5-bullet)

##### Часть 3. Ответы на вопросы:
* [1. Зависимость между наличием детей и возвратом кредита в срок.](#6-bullet)
* [2. Зависимость между семейным положением и возвратом кредита в срок](#7-bullet)
* [3. Зависимость дохода и возвратом кредита в срок.](#8-bullet)
* [4. Влияние целей кредита на его возврат в срок?.](#9-bullet)

##### Часть 4. Выводы :
* [1. Общий вывод.](#10-bullet)




<a id='1-bullet'></a>

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов.

Подробно разберём, какие в df столбцы и какую информацию они содержат:

children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

Тип данных у каждого столбца -  int64, float64, int64, object, int64, object, int64, object, object, int64, float64, object соответственно.

Вывод:
Каждая строка таблицы содержит информацию о человеке, которая влияет на платежеспособность клиента. Несколько проблем, которые нужно решать: пропуски, некорректные данные, некорректные типы данных, дубликаты. Для проверки рабочих гипотез особенно ценны столбцы  children, family_status,total_income и debt.

### Шаг 2. Предобработка данных

<a id='2-bullet'></a>

### Обработка пропусков

Из предыдущей таблицы видно, что есть часть пустых значений в столбцах days_employed и total_income. Пустые значения свидетельствуют, что для некоторых клиентов доступна не вся информация, в нашем случае-это общий трудовой стаж и ежемесячный доход. Причины могут быть разные: либо если наш клиент студент, поэтому у него нет стажа, а родители его обеспечивают, либо  пенсионер, который живет в хосписе, либо же сотрудник, конфиденциальные данные которого банк не хочет разглашать. Возможно данные не были введены из-за человеческого фактора. Каждый отдельный случай необходимо разобрать и выявить причину.

Заменим пропущенные значения в столбцах. После этой операции нужно убедиться, что таблица больше не содержит пропусков.

Заменим пропущенные значения в столбце 'days_employed' на 'unknown', так как значение данного столбца не влияют на наше исследование.
Неизвестные значения ежемесячного дохода составляют 10% от всех значений, поэтому не учитывать пропущенные данные мы не можем. 
Заменем пропущенные значения в столбце total_income на среднее значение ежемесячного дохода клиентов в зависимости от вида заработка.

In [3]:
df['days_employed']=df['days_employed'].fillna('unknown')

In [4]:
income_list=df['income_type'].unique()

In [5]:
for i_list in income_list:
    df.loc[(df['income_type']==i_list) & (df['total_income'].isna()), 'total_income']=df.loc[df['income_type']==i_list, 'total_income'].median()

In [6]:
df.isnull().sum()


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Итак, мы заменили все пропуски соответствующими значениями.

<a id='3-bullet'></a>

### Замена типа данных

Анализируя данные, мы будем часто использовать столбецы total_income и children типа float и object соответственно. Для удобства просмотра  и вычисления информации заменим float на int, object на int . На результат анализа изменения не повлияют , так как разница несущественна. 
Также смущает столбец days_employed, но использовать информацию этого столбца мы не будем , поэтому изменять тип данных будет лишней работой.

In [7]:
df['total_income']=df['total_income'].astype('int')

In [8]:
df['children']=df['children'].astype('int')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null object
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(6)
memory usage: 2.0+ MB


### Вывод

Мы заменили тип данных в одном столбце.

<a id='4-bullet'></a>

### Обработка дубликатов

Необходимо установить наличие дубликатов. Посмотрим, в каких столбцах могли быть ошибки при вводе.

In [10]:
df['income_type'].value_counts() 

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [11]:
df['family_status'].value_counts() 

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [12]:
df['education'].value_counts() 

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [13]:
df['gender'].value_counts() 

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [14]:
df['children'].value_counts() 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [15]:
df[df['children']==20].tail(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16795,20,-2530.54,29,среднее,1,женат / замужем,0,M,компаньон,0,95454,недвижимость
16886,20,-957.251,33,высшее,0,женат / замужем,0,F,сотрудник,0,86656,сделка с подержанным автомобилем
17093,20,-1481.6,42,среднее,1,женат / замужем,0,F,сотрудник,0,73069,получение дополнительного образования
17134,20,-1399.56,46,среднее,1,женат / замужем,0,M,компаньон,0,136446,покупка жилья для семьи
17286,20,unknown,50,высшее,0,в разводе,3,F,сотрудник,0,142594,покупка коммерческой недвижимости
17509,20,-461.94,31,среднее,1,Не женат / не замужем,4,F,сотрудник,0,90937,жилье
18476,20,-4686.11,36,среднее,1,женат / замужем,0,M,госслужащий,0,258005,на покупку подержанного автомобиля
18493,20,-121,49,среднее,1,женат / замужем,0,F,компаньон,0,306808,строительство собственной недвижимости
18756,20,-2334.69,51,среднее,1,женат / замужем,0,F,сотрудник,0,224637,автомобиль
18890,20,-904.259,37,среднее,1,женат / замужем,0,F,компаньон,1,212811,на покупку подержанного автомобиля


In [16]:
df[df['children']==-1].tail(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
11109,-1,-3438.46,59,среднее,1,женат / замужем,0,F,сотрудник,0,219874,операции с жильем
11268,-1,-1048.78,30,высшее,0,гражданский брак,1,F,компаньон,0,321603,на покупку автомобиля
11617,-1,-370.827,27,среднее,1,женат / замужем,0,F,компаньон,0,164591,покупка жилья для сдачи
12304,-1,-8493.1,61,неоконченное высшее,2,женат / замужем,0,M,сотрудник,0,315006,недвижимость
13786,-1,unknown,42,среднее,1,Не женат / не замужем,4,M,компаньон,0,172357,автомобиль
14357,-1,-268.337,23,среднее,1,в разводе,3,F,компаньон,0,92257,покупка жилой недвижимости
15147,-1,-9851.18,44,среднее,1,в разводе,3,F,сотрудник,0,110990,свой автомобиль
15435,-1,-3614.22,40,высшее,0,гражданский брак,1,F,сотрудник,0,98127,сыграть свадьбу
16129,-1,-457.862,33,среднее,1,женат / замужем,0,F,сотрудник,1,149641,автомобиль
16265,-1,-2802.22,40,высшее,0,женат / замужем,0,M,сотрудник,0,111984,покупка жилья


In [17]:
df['purpose'].value_counts() 

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Мы получили, что в столбцах 'purpose','gender', 'children' и 'education' существуют ошибки. 
Выделяются семьи с 20-ю детьми, так самые многодетные клиенты после семей с 20-ю детьми, имеют 5 детей. Более того , некоторым клиентам меньше 40 лет, а у них уже есть 20 детей. Такое возможно в реальном мире, но маловероятно, поэтому целесообразно удалить из базы этих клиентов, чтобы не испортить результаты исследований. Также существуют клиенты в столбце children со значением -1. Вероятнее всего, это клиенты потерявшие детей, поэтому заменем значение -1 на 0.
В столбце 'education' приведем все значения к нижнему регистру.
На этом этапе найдем и удалим все дубликаты в таблице. 

In [18]:
df.drop(df[df['children'] ==20].index, inplace=True)

In [19]:
df['children']=df['children'].replace(-1, 0)

In [20]:
df['education'].str.lower()

0         высшее
1        среднее
2        среднее
3        среднее
4        среднее
          ...   
21520    среднее
21521    среднее
21522    среднее
21523    среднее
21524    среднее
Name: education, Length: 21449, dtype: object

In [21]:
df.duplicated().sum()

54

In [22]:
df=df.drop_duplicates().reset_index(drop= True)

In [23]:
df.duplicated().sum()

0

### Вывод

Мы убрали часть дубликатов, но другая часть могла появиться из-за сбоя записи данных. Разбиремся с причинами появления такого «информационного мусора».

### Лемматизация

Выше мы нашли, что один из проблемных столбцов это purpose. Проблема заключается в том , что компьютер выдает большое количество целей, но на самом деле одна и та же цель записана разными словами. Сделаем так, чтобы синонимичные цели были записаны одним и тем же выражением.

Для этого подключим соответствующую библиотеку, которая поможет обойти разнообразие русскоязычных форм слов. Приведем слова в начальную форму.

In [24]:
from pymystem3 import Mystem
m = Mystem()


In [25]:
def lemataiz(string):
    lemmas = ' '.join(m.lemmatize(string))
    return lemmas

In [26]:
df['lemmas']=df['purpose'].apply(lemataiz)

In [27]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,-4024.8,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,-5623.42,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n


Теперь составим список ключевых слов.

In [28]:
df['lemmas'].value_counts() 

автомобиль \n                                    970
свадьба \n                                       792
на   проведение   свадьба \n                     769
сыграть   свадьба \n                             765
операция   с   недвижимость \n                   674
покупка   коммерческий   недвижимость \n         659
покупка   жилье   для   сдача \n                 651
операция   с   жилье \n                          648
операция   с   коммерческий   недвижимость \n    646
покупка   жилье \n                               643
жилье \n                                         642
покупка   жилье   для   семья \n                 637
недвижимость \n                                  632
строительство   собственный   недвижимость \n    629
операция   со   свой   недвижимость \n           626
строительство   жилой   недвижимость \n          624
покупка   свой   жилье \n                        620
покупка   недвижимость \n                        619
строительство   недвижимость \n               

In [29]:
words=['автомобиль', 'свадьба', 'недвижимость', 'жилье', 'образование']

Теперь проверим, есть ли одно из этих слов в цели, если да, то запишем в новый столбец уже унифицированную цель.

In [30]:
def istina(row):
    for word in words:
        if word in row:
            return word
        

In [31]:
df['true_purpose']=df['lemmas'].apply(istina)

In [32]:
df['true_purpose'].value_counts()

недвижимость    6332
жилье           4446
автомобиль      4292
образование     3999
свадьба         2326
Name: true_purpose, dtype: int64

In [33]:
df['true_purpose']=df['true_purpose'].replace('жилье','недвижимость')

In [34]:
df['true_purpose'].value_counts()

недвижимость    10778
автомобиль       4292
образование      3999
свадьба          2326
Name: true_purpose, dtype: int64

In [35]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,true_purpose
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,недвижимость
1,1,-4024.8,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,автомобиль
2,0,-5623.42,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,недвижимость
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба


Теперь удалим ненужные столбцы и проверим на наличие дубликатов.

In [36]:
df = df.drop(['purpose', 'lemmas'], axis=1)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21395 entries, 0 to 21394
Data columns (total 12 columns):
children            21395 non-null int64
days_employed       21395 non-null object
dob_years           21395 non-null int64
education           21395 non-null object
education_id        21395 non-null int64
family_status       21395 non-null object
family_status_id    21395 non-null int64
gender              21395 non-null object
income_type         21395 non-null object
debt                21395 non-null int64
total_income        21395 non-null int64
true_purpose        21395 non-null object
dtypes: int64(6), object(6)
memory usage: 2.0+ MB


In [38]:
df.duplicated().sum()

287

In [39]:
df=df.drop_duplicates().reset_index(drop= True)

In [40]:
df.duplicated().sum()

0

### Вывод

Мы стандартизировали цели для кредита и удалили дубликаты. Теперь мы можем свободно пользоваться таблицей.

<a id='5-bullet'></a>

### Категоризация данных

В дальнейшем мы будем изучать зависимость между уровнем дохода и возвратом кредита в срок, поэтому разделим клиентов на 3 категории: бедные, средний класс и богатые.

In [41]:
df['total_income'].min()

20667

In [42]:
df['total_income'].max()

2265604

In [43]:
def bogat_beden(row):
    income = row[10]
    if income <= 100000:
        return 'бедный'
    elif income >=1000000:
        return 'богатый'
    else:
        return 'средний достаток'



In [44]:
df['income']=df.apply(bogat_beden, axis=1)

In [45]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,true_purpose,income
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,средний достаток
1,1,-4024.8,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,средний достаток
2,0,-5623.42,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,средний достаток
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,средний достаток
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,средний достаток


Чтобы провести данное исследование, нам понадобится не вся информация из таблицы df, поэтому для удобства мы создали новую таблицу, которая состоит только из необходимых нам столбцов.

In [46]:
df_new=df[['children','family_status','family_status_id', 'debt', 'total_income', 'true_purpose', 'income']]
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21108 entries, 0 to 21107
Data columns (total 7 columns):
children            21108 non-null int64
family_status       21108 non-null object
family_status_id    21108 non-null int64
debt                21108 non-null int64
total_income        21108 non-null int64
true_purpose        21108 non-null object
income              21108 non-null object
dtypes: int64(4), object(3)
memory usage: 1.1+ MB


### Вывод

Мы классифицировали данные столбца 'total_income' на 3 категория для проведения более точного анализа. Также мы создали новую таблицу df_new на базе данной таблицы df.

### Шаг 3. Ответьте на вопросы

<a id='6-bullet'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [47]:
df_new.groupby('debt')['children'].sum()

debt
0    9106
1     929
Name: children, dtype: int64

Из первой таблицы видно, что чем больше детей у клиентов, тем больше вероятность того, что долг будет погашен вовремя.
Давайте проверим, настолько такое вывод верен.

In [48]:
df_new.groupby('children').agg({'debt': ['count', 'sum']})

debt      
          count   sum
children             
0         13929  1062
1          4759   444
2          2042   194
3           329    27
4            40     4
5             9     0

In [49]:
df_child=df_new.groupby('children')['debt'].sum()/df_new.groupby('children')['debt'].count()*100

In [50]:
df_child.sort_values()

children
5     0.000000
0     7.624381
3     8.206687
1     9.329691
2     9.500490
4    10.000000
Name: debt, dtype: float64

Рассмотрим две вышеизложенные таблицы. Мы видим  большой разброс значений у клиентов с 5-ю и 4-мя детьми. Предлагаю объединить значения 4-5 детей, получим средне значение по этому промежутку.

In [51]:
df['children']=df['children'].replace(5 , '4-5')

In [52]:
df['children']=df['children'].replace(4 , '4-5')

In [53]:
df.groupby('children').agg({'debt': ['count', 'sum']})

debt      
          count   sum
children             
0         13929  1062
1          4759   444
2          2042   194
3           329    27
4-5          49     4

In [54]:
df_child_1=df.groupby('children')['debt'].sum()/df.groupby('children')['debt'].count()*100

In [55]:
df_child_1.sort_values()

children
0      7.624381
4-5    8.163265
3      8.206687
1      9.329691
2      9.500490
Name: debt, dtype: float64

In [56]:
all_with_debt=df_new.groupby('debt')['debt'].count()

In [57]:
df_child=df.groupby('children')['debt'].sum()/all_with_debt[1]*100

In [58]:
df_child.sort_values()

children
4-5     0.231080
3       1.559792
2      11.207395
1      25.649913
0      61.351820
Name: debt, dtype: float64

Итак, по двум последним таблицам мы можем сделать вывод, что именно люди без детей больше берут кредитов, при этом большая часть из них выплачивают кредит в срок. Однако уже с появлением 1 и 2 ребенка ситуация несколько меняется: кредитов берут меньше, но имеет задолженности по кредиту больше человек. Как видно из таблицы df_child_1 многодетные семьи редко берут кредиты и большая часть таких клиентов выплачивают долг в срок.  

### Вывод

Мы получили нелинейный вывод. Люди без детей и многодетные семьи имеют меньше долгов по кредитам, чем клиенты с 1-2 детьми. Возможно , это обусловлено тем , что с появлением 3 ребенка люди боятся попасть в долговую яму, так как на них лежит большая ответственность.

<a id='7-bullet'></a>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [59]:
resultat=df_new.groupby('family_status')['debt'].sum()/df_new.groupby('family_status')['debt'].count()*100

In [60]:
resultat.sort_values()

family_status
вдовец / вдова           6.680806
в разводе                7.052897
женат / замужем          7.673821
гражданский брак         9.331071
Не женат / не замужем    9.816613
Name: debt, dtype: float64

Таблица показывает процентное соотношение клиентов с долгом к общему числу клиентов по каждой категории.

In [61]:
resultat=df_new.groupby('family_status')['debt'].sum()/all_with_debt[1]*100

In [62]:
resultat.sort_values()

family_status
вдовец / вдова            3.639515
в разводе                 4.852686
Не женат / не замужем    15.771231
гражданский брак         22.241479
женат / замужем          53.495090
Name: debt, dtype: float64

Таблица показывает процентное соотношение клиентов с долгом по каждой категории к общему числу клиентов.

### Вывод

Итак, из таблиц видно, что именно одинокие люди (вдовец/вдова или в разводе) меньше берут кредитов и быстрее его выплачивают. Обратная ситуация обстоит с людьми,которые состоят в браке (женат/замужем, гражданский брак) они больше берут кредитов и большая часть людей их не выплачивает. 

<a id='8-bullet'></a>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [63]:
df_new.groupby('income')['debt'].sum()/df_new.groupby('income')['debt'].count()*100

income
бедный              7.951482
богатый             8.000000
средний достаток    8.267693
Name: debt, dtype: float64

Из предыдущей таблицы мы узнаем, что в каждой категории 8% клиентов от общего числа соответствующих категорий не выплачивают долги в срок.

In [64]:
df_new.groupby('income')['debt'].sum()/all_with_debt[1]*100

income
бедный              20.450607
богатый              0.115540
средний достаток    79.433853
Name: debt, dtype: float64

При этом большая часть клиентов с невыплаченными долгами в срок от общего числа относятся к среднему классу или являются бедными, как видно из второй таблицы.

In [65]:
df_new.groupby('debt').agg({'total_income': ['min', 'max', 'mean', 'median']})

total_income                                
              min      max           mean  median
debt                                             
0           21205  2265604  166031.725757  142752
1           20667  2200852  161167.071057  142594

А из последней таблицы мы видим, что по всем параметрам уровень дохода клиентов без долга незначительно , но больше уровня дохода клиентов с кредитом.

### Вывод

Таким образом, мы можем сказать, что люди среднего класса больше берут кредитов по сравнению с бедными и состоятельными людьми, хотя 8% каждой категории не выплачивают кредит срок. При этом стоит заметить , что среднестатистические данные выдают, лучшие показатели дохода у людей без долгов, чем у людей с задолженностью по кредиту. Поэтому мы можем сделать вывод, что чем богаче человек , тем меньше вероятность того, что он вернет долг во время, хотя эта оценка не велика.

<a id='9-bullet'></a>

- Как разные цели кредита влияют на его возврат в срок?

In [66]:
resultat=df_new.groupby('true_purpose')['debt'].sum()/df_new.groupby('true_purpose')['debt'].count()*100

In [67]:
resultat.sort_values()

true_purpose
недвижимость    7.362949
свадьба         7.925509
образование     9.322890
автомобиль      9.387468
Name: debt, dtype: float64

Мы нашли долю клиентов с непогашенным долгом по каждой категории. 

### Вывод

Из таблицы видно, что клиенты, взявшие кредит на операции с недвижимостью охотнее выплачивают кредиты в срок, от них не далеко ушли клиенты , взявшие кредит на свадьбу. Хуже всего с задачей вернуть кредитит справились клиенты , которые купили машину на деньги банка. Однако если смотреть на ситуацию в общем, то разница между процентами каждой категории не столь велики, поэтому можно сделать вывод, что цели кредита не влияют на погашение кридита в срок. 

<a id='10-bullet'></a>

### Шаг 4. Общий вывод

Многие факторы влияют на кредитоспособность клиента. На анализе данной нам таблице мы выявили, что уровень дохода, количество детей и семейное положение влияют выплату долга в срок.